In [12]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import compose, pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn import compose, pipeline
from sklearn.model_selection import KFold
from typing import Optional
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import roc_auc_score, make_scorer, r2_score
from pandas import DataFrame, Series
import numpy as np
kf = KFold(n_splits=5, shuffle=True, random_state=42)
import xgboost as xgb
from xgboost import XGBRegressor
import optuna
from optuna import Trial
from optuna import create_study
from sklearn import compose
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, Ridge, LinearRegression, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
import plotly.express as px
#website help from 
#https://medium.com/@walter_sperat/using-optuna-with-sklearn-the-right-way-part-1-6b4ad0ab2451

In [13]:
df = pd.read_csv('dataframes/df_normal_quality.csv', index_col=0)
y = df.SalePrice
X = df.drop(['PID', 'SalePrice'], axis =1).copy()

In [14]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
#from category_encoders import WOEEncoder

def instantiate_ordinal_encoder(trial: Trial)-> OrdinalEncoder:
    params = {
        'handle_unknown': "use_encoded_value", 
        'unknown_value': -1
    }
    
    return OrdinalEncoder(**params)

# def instantiate_onehot_encoder(trial: Trial)-> OneHotEncoder:
#     params = {
#         'handle_unknown': 'ignore',
#         'drop': trial.suggest_categorical('drop', [None, 'first'])
#     }
    
#     return OneHotEncoder(**params)

def instantiate_onehot_encoder(trial: Trial)-> OneHotEncoder:
    params = {
        'handle_unknown': 'ignore',
        'drop': None
    }
    
    return OneHotEncoder(**params)
    
Encoder = (
    OrdinalEncoder |
    OneHotEncoder 
    )

def instantiate_encoder (trial : Trial) -> Encoder:
    encoding_method = trial.suggest_categorical(
        'encoding_method', ['ordinal', 'onehot'])
    if encoding_method =='ordinal':
        encoder = instantiate_ordinal_encoder(trial)
    elif encoding_method =='onehot':
        encoder = instantiate_onehot_encoder(trial)
    
    return encoder

from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler

Scaler = (
  StandardScaler |
  MinMaxScaler |
  MaxAbsScaler |
  RobustScaler
)

def instantiate_scaler(trial : Trial) -> Scaler:
    method = trial.suggest_categorical(
    'scaling_method', ['standard', 'minmax', 'maxabs', 'robust']
    )
    if method=='standard':
        scaler = StandardScaler()
    elif method=='minmax':
        scaler = MinMaxScaler()
    elif method=='maxabs':
        scaler = MaxAbsScaler()
    elif method=='robust':
        scaler = RobustScaler()
        
    return scaler

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

def instantiate_processor(trial : Trial, 
                          numerical_columns : list[str], 
                          categorical_columns : list[str]) -> ColumnTransformer:
    
    numerical_pipeline = instantiate_scaler(trial)
    categorical_pipeline = instantiate_encoder(trial)
   # numerical_pipeline = StandardScaler()
    #categorical_pipeline = OneHotEncoder(handle_unknown="ignore", drop='first')
    
#     numerical_pipeline = instantiate_numerical_pipeline(trial)
#     categorical_pipeline = instantiate_categorical_pipeline(trial)
    
    processor = ColumnTransformer([
        ('numerical_pipeline', numerical_pipeline, numerical_columns),
        ('categorical_pipeline', categorical_pipeline, categorical_columns)
    ])
    return processor

def instantiate_processor_ordinal(trial : Trial, 
                          numerical_columns : list[str], 
                          categorical_columns : list[str]) -> ColumnTransformer:
    
    numerical_pipeline = instantiate_scaler(trial)
    categorical_pipeline = instantiate_ordinal_encoder(trial)

    processor = ColumnTransformer([
        ('numerical_pipeline', numerical_pipeline, numerical_columns),
        ('categorical_pipeline', categorical_pipeline, categorical_columns)
    ])
    return processor

# Lasso

In [5]:
def instantiate_lasso(trial : Trial) -> Lasso:
    params = {
        "alpha": trial.suggest_float("alpha", .00001, .001, log=True)
    }

    return Lasso(**params)

def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_lasso(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_lasso(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [6]:
from optuna import create_study

study_lasso = create_study(study_name='optimization', direction='maximize')

study_lasso.optimize(lambda trial: objective_lasso(trial, X, y), n_trials=200)

[I 2024-08-05 13:10:45,124] A new study created in memory with name: optimization
[I 2024-08-05 13:10:45,834] Trial 0 finished with value: 0.9377408902056376 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'ordinal', 'alpha': 0.00021761272106793047}. Best is trial 0 with value: 0.9377408902056376.
[I 2024-08-05 13:10:46,676] Trial 1 finished with value: 0.9413755633135569 and parameters: {'scaling_method': 'robust', 'encoding_method': 'ordinal', 'alpha': 7.120139155776975e-05}. Best is trial 1 with value: 0.9413755633135569.
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.001e+00, tolerance: 2.383e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: Convergen

[I 2024-08-05 13:11:00,743] Trial 9 finished with value: 0.9403413643658739 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'ordinal', 'alpha': 8.228755975283896e-05}. Best is trial 4 with value: 0.9498808856885278.
[I 2024-08-05 13:11:01,340] Trial 10 finished with value: 0.9500310202186955 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'alpha': 0.0004622679904209737}. Best is trial 10 with value: 0.9500310202186955.
[I 2024-08-05 13:11:01,926] Trial 11 finished with value: 0.9496909663302651 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'alpha': 0.0005866965216905682}. Best is trial 10 with value: 0.9500310202186955.
[I 2024-08-05 13:11:02,625] Trial 12 finished with value: 0.9502920522179448 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'alpha': 0.00028994438358483456}. Best is trial 12 with value: 0.9502920522179448.
[I 2024-08-05 13:11:03,260] Trial 13 finished with value: 0.95

[I 2024-08-05 13:11:32,138] Trial 41 finished with value: 0.9499617870592238 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'alpha': 0.00036980618575964424}. Best is trial 39 with value: 0.9506437403516307.
[I 2024-08-05 13:11:33,168] Trial 42 finished with value: 0.950643686758009 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'alpha': 0.00022393039632001444}. Best is trial 39 with value: 0.9506437403516307.
[I 2024-08-05 13:11:34,448] Trial 43 finished with value: 0.9504773958915441 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'alpha': 0.00013771781002077459}. Best is trial 39 with value: 0.9506437403516307.
[I 2024-08-05 13:11:36,121] Trial 44 finished with value: 0.9502227517491331 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'alpha': 9.623490361532898e-05}. Best is trial 39 with value: 0.9506437403516307.
[I 2024-08-05 13:11:36,994] Trial 45 finished with value: 0.950697

[I 2024-08-05 13:11:57,168] Trial 69 finished with value: 0.95081254796895 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0003057831898664838}. Best is trial 69 with value: 0.95081254796895.
[I 2024-08-05 13:11:57,824] Trial 70 finished with value: 0.9504796276682276 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0005171764622919398}. Best is trial 69 with value: 0.95081254796895.
[I 2024-08-05 13:11:58,547] Trial 71 finished with value: 0.9508119303189823 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0003034227163257451}. Best is trial 69 with value: 0.95081254796895.
[I 2024-08-05 13:11:59,239] Trial 72 finished with value: 0.9508109239091638 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.00031417934239634006}. Best is trial 69 with value: 0.95081254796895.
[I 2024-08-05 13:12:00,025] Trial 73 finished with value: 0.9507490204649448 

[I 2024-08-05 13:12:25,116] Trial 105 finished with value: 0.9508088498954933 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0002932326932749235}. Best is trial 69 with value: 0.95081254796895.
[I 2024-08-05 13:12:26,999] Trial 106 finished with value: 0.9488862494084837 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 5.391269543292068e-05}. Best is trial 69 with value: 0.95081254796895.
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.402e+00, tolerance: 2.383e-02
  model = cd_fast.enet_coordinate_descent(
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of ite

[I 2024-08-05 13:12:49,744] Trial 129 finished with value: 0.9505105001380943 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.000205504745762836}. Best is trial 69 with value: 0.95081254796895.
[I 2024-08-05 13:12:50,363] Trial 130 finished with value: 0.9504297958041865 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0005357952284947135}. Best is trial 69 with value: 0.95081254796895.
[I 2024-08-05 13:12:51,122] Trial 131 finished with value: 0.9508114743586343 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.00029921861800320495}. Best is trial 69 with value: 0.95081254796895.
[I 2024-08-05 13:12:51,858] Trial 132 finished with value: 0.9508066307343686 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.00033511931994313653}. Best is trial 69 with value: 0.95081254796895.
[I 2024-08-05 13:12:52,546] Trial 133 finished with value: 0.9507532281

[I 2024-08-05 13:13:17,670] Trial 165 finished with value: 0.9507306203636887 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.00039541674802286245}. Best is trial 69 with value: 0.95081254796895.
[I 2024-08-05 13:13:18,413] Trial 166 finished with value: 0.9508083349616603 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0002925474152905969}. Best is trial 69 with value: 0.95081254796895.
[I 2024-08-05 13:13:18,948] Trial 167 finished with value: 0.9417207815146533 and parameters: {'scaling_method': 'robust', 'encoding_method': 'ordinal', 'alpha': 0.0002492496982466766}. Best is trial 69 with value: 0.95081254796895.
[I 2024-08-05 13:13:19,648] Trial 168 finished with value: 0.9507862189872942 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0003565653117688196}. Best is trial 69 with value: 0.95081254796895.
[I 2024-08-05 13:13:20,444] Trial 169 finished with value: 0.950767950

In [7]:
print(study_lasso.best_value)
print(study_lasso.best_params)

0.95081254796895
{'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0003057831898664838}


In [ ]:
trial_number = []
score = []
parameters = []
optuna_dict = {}
trials = range(200)

for trial in trials:
    trial_number.append(study_lasso.get_trials()[trial].number)
    score.append(study_lasso.get_trials()[trial].value)
    parameters.append(list(study_lasso.get_trials()[trial].params.items()))

optuna_dict['Trial'] = trial_number
optuna_dict['Score'] = score
optuna_dict['Parameters'] = parameters

optuna_lasso = pd.DataFrame.from_dict(optuna_dict)
optuna_lasso

In [ ]:
pd.set_option('display.max_colwidth', None) 
optuna_lasso.Parameters

In [ ]:
 optuna_lasso['alpha'] = optuna_lasso.Parameters.apply(lambda x: x[-1][1])

In [ ]:
 optuna_lasso['encoding'] = optuna_lasso.Parameters.apply(lambda x: x[1][1])

In [ ]:
optuna_lasso['scaling'] = optuna_lasso.Parameters.apply(lambda x: x[0][1])

In [ ]:
optuna_lasso.to_csv('dataframes/optuna/optuna_lasso.csv')

In [ ]:
fig = px.scatter(optuna_lasso,
                 x="Trial", 
                 y="Score", 
                 hover_data=['Parameters'],
                labels = {'Trial': 'Optuna Trial', 'Score': 'Score (R-Squared)'},
                title = 'Lasso tuned with Optuna after 200 trials  <br><sup> Best Score: 0.9508126</sup>')

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)

fig.show()

fig.write_image("images/lasso_optuna.png")

In [ ]:

fig = px.scatter(optuna_lasso,
                 x="Trial", 
                 y="Score", 
                 hover_data=['Parameters'],
                 color = 'scaling',
                labels = {'Trial': 'Optuna Trial', 'Score': 'Score (R-Squared)'},
                title = 'Lasso tuned with Optuna after 200 trials  <br><sup> Best Score: 0.9508126</sup>')

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)

fig.show()

fig.write_image("images/lasso_optuna_scaling.png")

In [ ]:

fig = px.scatter(optuna_lasso,
                 x="Trial", 
                 y="Score", 
                 hover_data=['Parameters'],
                 color = 'alpha',
                labels = {'Trial': 'Optuna Trial', 'Score': 'Score (R-Squared)'},
                title = 'Lasso tuned with Optuna after 200 trials  <br><sup> Best Score: 0.9508126</sup>')

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)

fig.show()

fig.write_image("images/lasso_optuna_alpha.png")

In [ ]:

fig = px.scatter(optuna_lasso,
                 x="Trial", 
                 y="Score", 
                 hover_data=['Parameters'],
                 color = 'encoding',
                labels = {'Trial': 'Optuna Trial', 'Score': 'Score (R-Squared)'},
                title = 'Lasso tuned with Optuna after 200 trials  <br><sup> Best Score: 0.9508126</sup>')

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)

fig.show()

fig.write_image("images/lasso_optuna_encoding.png")

# Ridge

In [81]:
def instantiate_ridge(trial : Trial) -> Ridge:
    params = {
        "alpha": trial.suggest_float("alpha", 1, 100, log=True)
    }

    return Ridge(**params)


def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_ridge(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_ridge(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [82]:
from optuna import create_study

study_ridge = create_study(study_name='optimization', direction='maximize')

study_ridge.optimize(lambda trial: objective_ridge(trial, X, y), n_trials=200)

[I 2024-08-09 11:54:36,164] A new study created in memory with name: optimization
[I 2024-08-09 11:54:36,598] Trial 0 finished with value: 0.8588577289989221 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'onehot', 'alpha': 86.23334938940869}. Best is trial 0 with value: 0.8588577289989221.
[I 2024-08-09 11:54:37,022] Trial 1 finished with value: 0.9490698690331645 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'alpha': 5.597117642173478}. Best is trial 1 with value: 0.9490698690331645.
[I 2024-08-09 11:54:37,439] Trial 2 finished with value: 0.9459098228632861 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'onehot', 'alpha': 1.413432947101729}. Best is trial 1 with value: 0.9490698690331645.
[I 2024-08-09 11:54:37,831] Trial 3 finished with value: 0.9402483410105326 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'alpha': 9.777004058906657}. Best is trial 1 with value: 0.9490698690331645.
[I 2024-

[I 2024-08-09 11:54:53,040] Trial 36 finished with value: 0.9500472709720466 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 21.503935588119514}. Best is trial 36 with value: 0.9500472709720466.
[I 2024-08-09 11:54:53,467] Trial 37 finished with value: 0.931226831270205 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'alpha': 21.960083831602045}. Best is trial 36 with value: 0.9500472709720466.
[I 2024-08-09 11:54:53,853] Trial 38 finished with value: 0.9051880060863151 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'ordinal', 'alpha': 34.409472742206226}. Best is trial 36 with value: 0.9500472709720466.
[I 2024-08-09 11:54:54,308] Trial 39 finished with value: 0.9491212138948658 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 60.24853738368133}. Best is trial 36 with value: 0.9500472709720466.
[I 2024-08-09 11:54:54,801] Trial 40 finished with value: 0.9498112008181104 and 

[I 2024-08-09 11:55:09,676] Trial 72 finished with value: 0.9500421784604953 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 20.314081662329183}. Best is trial 51 with value: 0.9500479784981619.
[I 2024-08-09 11:55:10,203] Trial 73 finished with value: 0.9499109232932454 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 34.10794152360055}. Best is trial 51 with value: 0.9500479784981619.
[I 2024-08-09 11:55:10,657] Trial 74 finished with value: 0.949953058627586 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 14.975144156472288}. Best is trial 51 with value: 0.9500479784981619.
[I 2024-08-09 11:55:11,001] Trial 75 finished with value: 0.9222799410119258 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'onehot', 'alpha': 27.401311037837974}. Best is trial 51 with value: 0.9500479784981619.
[I 2024-08-09 11:55:11,379] Trial 76 finished with value: 0.9500483603405957 and p

[I 2024-08-09 11:55:26,568] Trial 108 finished with value: 0.9362900579514374 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'alpha': 14.959271989705373}. Best is trial 76 with value: 0.9500483603405957.
[I 2024-08-09 11:55:26,978] Trial 109 finished with value: 0.9500380440003781 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 19.732529025699257}. Best is trial 76 with value: 0.9500483603405957.
[I 2024-08-09 11:55:27,419] Trial 110 finished with value: 0.9485573306684293 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 2.8255857816693273}. Best is trial 76 with value: 0.9500483603405957.
[I 2024-08-09 11:55:28,043] Trial 111 finished with value: 0.9500483598474784 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 22.3724801415504}. Best is trial 76 with value: 0.9500483603405957.
[I 2024-08-09 11:55:28,544] Trial 112 finished with value: 0.9500483101641312 

[I 2024-08-09 11:55:42,804] Trial 144 finished with value: 0.9500266998161994 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 26.655247088721474}. Best is trial 76 with value: 0.9500483603405957.
[I 2024-08-09 11:55:43,292] Trial 145 finished with value: 0.950047253635604 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 21.496984215231638}. Best is trial 76 with value: 0.9500483603405957.
[I 2024-08-09 11:55:43,768] Trial 146 finished with value: 0.9500281430429313 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 18.730334598816818}. Best is trial 76 with value: 0.9500483603405957.
[I 2024-08-09 11:55:44,244] Trial 147 finished with value: 0.9499975766737496 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 16.78510754890285}. Best is trial 76 with value: 0.9500483603405957.
[I 2024-08-09 11:55:44,657] Trial 148 finished with value: 0.9496346231975282 

[I 2024-08-09 11:56:00,442] Trial 180 finished with value: 0.9500464161071367 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 23.61855654956772}. Best is trial 76 with value: 0.9500483603405957.
[I 2024-08-09 11:56:00,942] Trial 181 finished with value: 0.9500483590910239 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 22.36165924970174}. Best is trial 76 with value: 0.9500483603405957.
[I 2024-08-09 11:56:01,376] Trial 182 finished with value: 0.9500222134811231 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 18.261169166906186}. Best is trial 76 with value: 0.9500483603405957.
[I 2024-08-09 11:56:01,847] Trial 183 finished with value: 0.9500461983689439 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 21.14736314029406}. Best is trial 76 with value: 0.9500483603405957.
[I 2024-08-09 11:56:02,345] Trial 184 finished with value: 0.9500409023861944 a

In [83]:
print(study_ridge.best_value)
print(study_ridge.best_params)

0.9500483603405957
{'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 22.385505381094468}


In [84]:
trial_number = []
score = []
parameters = []
optuna_dict = {}
trials = range(200)

for trial in trials:
    trial_number.append(study_ridge.get_trials()[trial].number)
    score.append(study_ridge.get_trials()[trial].value)
    parameters.append(list(study_ridge.get_trials()[trial].params.items()))

optuna_dict['Trial'] = trial_number
optuna_dict['Score'] = score
optuna_dict['Parameters'] = parameters

optuna_ridge = pd.DataFrame.from_dict(optuna_dict)
optuna_ridge


,Trial,Score,Parameters
0,0,0.858858,"[(scaling_method, maxabs), (encoding_method, o..."
1,1,0.949070,"[(scaling_method, standard), (encoding_method,..."
2,2,0.945910,"[(scaling_method, maxabs), (encoding_method, o..."
3,3,0.940248,"[(scaling_method, minmax), (encoding_method, o..."
4,4,0.938135,"[(scaling_method, minmax), (encoding_method, o..."
...,...,...,...
195,195,0.950046,"[(scaling_method, robust), (encoding_method, o..."
196,196,0.950033,"[(scaling_method, robust), (encoding_method, o..."
197,197,0.941837,"[(scaling_method, robust), (encoding_method, o..."
198,198,0.950034,"[(scaling_method, robust), (encoding_method, o..."


In [85]:
optuna_ridge.to_csv('final_project/dataframes/optuna/optuna_ridge.csv')

# EN

In [38]:
def instantiate_en(trial : Trial) -> ElasticNet:
    params = {
        "alpha": trial.suggest_float("alpha", .00001, .001, log=True),
        'l1_ratio': trial.suggest_float('l1_ratio', .1, .7, log = True)
    }

    return ElasticNet(**params)



def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_en(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_en(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [39]:
from optuna import create_study

study_en = create_study(study_name='optimization', direction='maximize')

study_en.optimize(lambda trial: objective_en(trial, X, y), n_trials=200)

[I 2024-08-06 16:40:18,947] A new study created in memory with name: optimization
[I 2024-08-06 16:40:20,778] Trial 0 finished with value: 0.9485101642192975 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.00017960961266990318, 'l1_ratio': 0.20630385615556132}. Best is trial 0 with value: 0.9485101642192975.
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.023e-02, tolerance: 2.383e-02

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.695e-02, tolerance: 2.345e-02

C:\Users\Natal\anaconda3\Lib\si

[I 2024-08-06 16:40:39,579] Trial 14 finished with value: 0.9500306327022351 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'alpha': 0.0003108254054428836, 'l1_ratio': 0.4965878735239358}. Best is trial 11 with value: 0.9502867321604869.
[I 2024-08-06 16:40:40,525] Trial 15 finished with value: 0.9499463722900312 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'alpha': 0.0005430935681303572, 'l1_ratio': 0.23374363323917802}. Best is trial 11 with value: 0.9502867321604869.
[I 2024-08-06 16:40:42,173] Trial 16 finished with value: 0.9489528489046407 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'alpha': 0.00019627526780464626, 'l1_ratio': 0.28554361439751363}. Best is trial 11 with value: 0.9502867321604869.
[I 2024-08-06 16:40:44,488] Trial 17 finished with value: 0.948404286067279 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'alpha': 0.00022534430669881813, 'l1_ratio':

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.387e+00, tolerance: 2.383e-02

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.077e+00, tolerance: 2.345e-02

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.017e+00, tolerance: 2.415e-02

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_co

[I 2024-08-06 16:41:30,741] Trial 59 finished with value: 0.9417770448844351 and parameters: {'scaling_method': 'robust', 'encoding_method': 'ordinal', 'alpha': 0.000612964001328954, 'l1_ratio': 0.5389322507344019}. Best is trial 50 with value: 0.9508208059467489.
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.938e-02, tolerance: 2.383e-02

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.573e-02, tolerance: 2.345e-02

C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning:



[I 2024-08-06 16:41:53,919] Trial 85 finished with value: 0.9506384429283339 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0005920478873357701, 'l1_ratio': 0.382255468107673}. Best is trial 50 with value: 0.9508208059467489.
[I 2024-08-06 16:41:54,719] Trial 86 finished with value: 0.950681170863494 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0008345930709740543, 'l1_ratio': 0.27919863901652053}. Best is trial 50 with value: 0.9508208059467489.
[I 2024-08-06 16:41:57,251] Trial 87 finished with value: 0.9477660845622313 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 7.989641291005188e-05, 'l1_ratio': 0.3038798359737525}. Best is trial 50 with value: 0.9508208059467489.
C:\Users\Natal\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the sc

[I 2024-08-06 16:42:15,887] Trial 106 finished with value: 0.950759515984803 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0004684436157483229, 'l1_ratio': 0.5545449089380314}. Best is trial 50 with value: 0.9508208059467489.
[I 2024-08-06 16:42:16,412] Trial 107 finished with value: 0.9417631498991229 and parameters: {'scaling_method': 'robust', 'encoding_method': 'ordinal', 'alpha': 0.0006588219646910831, 'l1_ratio': 0.4464147045796851}. Best is trial 50 with value: 0.9508208059467489.
[I 2024-08-06 16:42:17,286] Trial 108 finished with value: 0.9506370895651178 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0005396422032500113, 'l1_ratio': 0.4207862216322988}. Best is trial 50 with value: 0.9508208059467489.
[I 2024-08-06 16:42:18,071] Trial 109 finished with value: 0.9505905876060666 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0008984620498475249, 'l1_ratio': 0.5113

[I 2024-08-06 16:42:40,718] Trial 131 finished with value: 0.9508062321400997 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0007184726037457723, 'l1_ratio': 0.387972305584297}. Best is trial 112 with value: 0.9508208209881921.
[I 2024-08-06 16:42:41,514] Trial 132 finished with value: 0.9508191613372933 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0007782771538996952, 'l1_ratio': 0.4107833998694829}. Best is trial 112 with value: 0.9508208209881921.
[I 2024-08-06 16:42:42,472] Trial 133 finished with value: 0.9507562257028519 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0006272733640074605, 'l1_ratio': 0.40839888414628134}. Best is trial 112 with value: 0.9508208209881921.
[I 2024-08-06 16:42:43,263] Trial 134 finished with value: 0.9507979483827459 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0007756318668635202, 'l1_ratio': 0.3

[I 2024-08-06 16:43:07,782] Trial 162 finished with value: 0.9508180713203134 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0006952497947441305, 'l1_ratio': 0.42428784042868956}. Best is trial 159 with value: 0.9508210183866526.
[I 2024-08-06 16:43:08,640] Trial 163 finished with value: 0.9508213344125327 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0007075093406413463, 'l1_ratio': 0.43312259030406497}. Best is trial 163 with value: 0.9508213344125327.
[I 2024-08-06 16:43:09,482] Trial 164 finished with value: 0.950820324881148 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.000691469693446336, 'l1_ratio': 0.43357685391542583}. Best is trial 163 with value: 0.9508213344125327.
[I 2024-08-06 16:43:10,326] Trial 165 finished with value: 0.9508212401053495 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0007040705360716474, 'l1_ratio': 0.

[I 2024-08-06 16:43:29,737] Trial 187 finished with value: 0.9492841293579634 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'alpha': 0.000998738814178994, 'l1_ratio': 0.3757054937946639}. Best is trial 163 with value: 0.9508213344125327.
[I 2024-08-06 16:43:30,540] Trial 188 finished with value: 0.9507150543368048 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0008859050839411457, 'l1_ratio': 0.4509591725216767}. Best is trial 163 with value: 0.9508213344125327.
[I 2024-08-06 16:43:31,359] Trial 189 finished with value: 0.9508069597057279 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0007145292858233232, 'l1_ratio': 0.39096287677285546}. Best is trial 163 with value: 0.9508213344125327.
[I 2024-08-06 16:43:32,265] Trial 190 finished with value: 0.950771486201457 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0005513026672421275, 'l1_ratio': 0.47

In [40]:
print(study_en.best_value)
print(study_en.best_params)

0.9508213344125327
{'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0007075093406413463, 'l1_ratio': 0.43312259030406497}


In [77]:
trial_number = []
score = []
parameters = []
optuna_dict = {}
trials = range(200)

for trial in trials:
    trial_number.append(study_en.get_trials()[trial].number)
    score.append(study_en.get_trials()[trial].value)
    parameters.append(list(study_en.get_trials()[trial].params.items()))

optuna_dict['Trial'] = trial_number
optuna_dict['Score'] = score
optuna_dict['Parameters'] = parameters

optuna_en = pd.DataFrame.from_dict(optuna_dict)
optuna_en


,Trial,Score,Parameters
0,0,0.948510,"[(scaling_method, robust), (encoding_method, o..."
1,1,0.941132,"[(scaling_method, minmax), (encoding_method, o..."
2,2,0.948555,"[(scaling_method, robust), (encoding_method, o..."
3,3,0.946307,"[(scaling_method, standard), (encoding_method,..."
4,4,0.947770,"[(scaling_method, minmax), (encoding_method, o..."
...,...,...,...
195,195,0.950819,"[(scaling_method, robust), (encoding_method, o..."
196,196,0.950380,"[(scaling_method, robust), (encoding_method, o..."
197,197,0.947736,"[(scaling_method, maxabs), (encoding_method, o..."
198,198,0.946343,"[(scaling_method, robust), (encoding_method, o..."


In [78]:
optuna_en.to_csv('final_project/dataframes/optuna/optuna_en.csv')

# XGB

In [15]:
def instantiate_xgb(trial : Trial) -> XGBRegressor:
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", .001, .1, log=True),
        "max_depth": trial.suggest_int("max_depth", 2, 4),
        "subsample": trial.suggest_float("subsample", 0.1, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 5)
    }

    return XGBRegressor(**params)

def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_xgb(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_xgb(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [16]:
from optuna import create_study

study_xgb = create_study(study_name='optimization', direction='maximize')

study_xgb.optimize(lambda trial: objective_xgb(trial, X, y), n_trials=200)

[I 2024-08-05 13:20:57,360] A new study created in memory with name: optimization
[I 2024-08-05 13:21:02,245] Trial 0 finished with value: -1.3807390749492907 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 342, 'learning_rate': 0.008172902216755243, 'max_depth': 3, 'subsample': 0.3185560399586813, 'colsample_bytree': 0.5546833595784315, 'min_child_weight': 5}. Best is trial 0 with value: -1.3807390749492907.
[I 2024-08-05 13:21:07,845] Trial 1 finished with value: -6.566164758051017 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'onehot', 'n_estimators': 629, 'learning_rate': 0.0013887345756673775, 'max_depth': 2, 'subsample': 0.21702580203559851, 'colsample_bytree': 0.6922750875266656, 'min_child_weight': 5}. Best is trial 0 with value: -1.3807390749492907.
[I 2024-08-05 13:21:09,357] Trial 2 finished with value: 0.9191512596134634 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'ordinal', 'n_estimators': 11

[I 2024-08-05 13:25:55,426] Trial 22 finished with value: 0.9472061154419261 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'n_estimators': 941, 'learning_rate': 0.09747896202812703, 'max_depth': 4, 'subsample': 0.8524179506388778, 'colsample_bytree': 0.6877472725543372, 'min_child_weight': 2}. Best is trial 17 with value: 0.950259793783033.
[I 2024-08-05 13:26:14,590] Trial 23 finished with value: 0.9492299712674155 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'n_estimators': 834, 'learning_rate': 0.02736697634770531, 'max_depth': 4, 'subsample': 0.7685179537541622, 'colsample_bytree': 0.8015692327621708, 'min_child_weight': 2}. Best is trial 17 with value: 0.950259793783033.
[I 2024-08-05 13:26:34,314] Trial 24 finished with value: 0.9516583209803287 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'n_estimators': 746, 'learning_rate': 0.0631234243662775, 'max_depth': 4, 'subsample': 0.6548225704496693, 

[I 2024-08-05 13:31:26,650] Trial 45 finished with value: 0.9456508223875938 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'onehot', 'n_estimators': 779, 'learning_rate': 0.04676198633847501, 'max_depth': 3, 'subsample': 0.11544329788064273, 'colsample_bytree': 0.5558648800344138, 'min_child_weight': 3}. Best is trial 24 with value: 0.9516583209803287.
[I 2024-08-05 13:31:34,642] Trial 46 finished with value: 0.9450357394572533 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'ordinal', 'n_estimators': 745, 'learning_rate': 0.03066823152909461, 'max_depth': 3, 'subsample': 0.21249589472646702, 'colsample_bytree': 0.6091602628362762, 'min_child_weight': 3}. Best is trial 24 with value: 0.9516583209803287.
[I 2024-08-05 13:31:45,211] Trial 47 finished with value: 0.9429021235505443 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'onehot', 'n_estimators': 705, 'learning_rate': 0.022616365871473868, 'max_depth': 3, 'subsample': 0.76600547062

[I 2024-08-05 13:36:32,082] Trial 68 finished with value: 0.9485866325511181 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 934, 'learning_rate': 0.09995519994062282, 'max_depth': 3, 'subsample': 0.3278228242377518, 'colsample_bytree': 0.517367020812364, 'min_child_weight': 2}. Best is trial 63 with value: 0.9526952468836074.
[I 2024-08-05 13:36:48,280] Trial 69 finished with value: 0.9515710293772194 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 913, 'learning_rate': 0.04887192831421519, 'max_depth': 3, 'subsample': 0.6586757261754846, 'colsample_bytree': 0.5186696195553557, 'min_child_weight': 1}. Best is trial 63 with value: 0.9526952468836074.
[I 2024-08-05 13:37:01,318] Trial 70 finished with value: 0.9515200780097333 and parameters: {'scaling_method': 'standard', 'encoding_method': 'ordinal', 'n_estimators': 906, 'learning_rate': 0.05061306373812772, 'max_depth': 3, 'subsample': 0.637295673

[I 2024-08-05 13:41:57,671] Trial 90 finished with value: 0.9483399280030056 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 811, 'learning_rate': 0.023061517025576343, 'max_depth': 3, 'subsample': 0.3524219730788454, 'colsample_bytree': 0.5358642246214007, 'min_child_weight': 2}. Best is trial 87 with value: 0.9531027787012614.
[I 2024-08-05 13:42:13,286] Trial 91 finished with value: 0.9529410901033251 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 953, 'learning_rate': 0.03777449230497327, 'max_depth': 3, 'subsample': 0.4959126936224624, 'colsample_bytree': 0.5406338068281275, 'min_child_weight': 2}. Best is trial 87 with value: 0.9531027787012614.
[I 2024-08-05 13:42:29,955] Trial 92 finished with value: 0.9537022219147874 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 950, 'learning_rate': 0.03857083391937703, 'max_depth': 3, 'subsample': 0.39302319

[I 2024-08-05 13:47:32,276] Trial 112 finished with value: 0.9521425362380509 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 974, 'learning_rate': 0.055143617587718734, 'max_depth': 3, 'subsample': 0.5199543460422458, 'colsample_bytree': 0.6233382722201515, 'min_child_weight': 2}. Best is trial 92 with value: 0.9537022219147874.
[I 2024-08-05 13:47:50,387] Trial 113 finished with value: 0.9524048940728875 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 979, 'learning_rate': 0.041948592747023734, 'max_depth': 3, 'subsample': 0.5056864934464852, 'colsample_bytree': 0.6391934937373278, 'min_child_weight': 2}. Best is trial 92 with value: 0.9537022219147874.
[I 2024-08-05 13:48:08,633] Trial 114 finished with value: 0.951611953651124 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 953, 'learning_rate': 0.038604453159630375, 'max_depth': 3, 'subsample': 0.4724

[I 2024-08-05 13:53:40,338] Trial 134 finished with value: 0.9481164026280716 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 435, 'learning_rate': 0.04725305655889264, 'max_depth': 3, 'subsample': 0.47425206286838784, 'colsample_bytree': 0.6076759053587119, 'min_child_weight': 2}. Best is trial 92 with value: 0.9537022219147874.
[I 2024-08-05 13:53:56,543] Trial 135 finished with value: 0.9517992551640387 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 929, 'learning_rate': 0.0667613476136499, 'max_depth': 3, 'subsample': 0.409897737507572, 'colsample_bytree': 0.5810302276267036, 'min_child_weight': 2}. Best is trial 92 with value: 0.9537022219147874.
[I 2024-08-05 13:54:03,543] Trial 136 finished with value: 0.9443247410245721 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'onehot', 'n_estimators': 333, 'learning_rate': 0.050772254318266694, 'max_depth': 3, 'subsample': 0.51083114

[I 2024-08-05 13:59:39,573] Trial 156 finished with value: 0.9520124410427891 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 1000, 'learning_rate': 0.04955875182141938, 'max_depth': 3, 'subsample': 0.526324542462333, 'colsample_bytree': 0.7028655856026025, 'min_child_weight': 2}. Best is trial 140 with value: 0.9545422235903054.
[I 2024-08-05 13:59:56,464] Trial 157 finished with value: 0.9499560450438641 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 947, 'learning_rate': 0.037173106555798474, 'max_depth': 3, 'subsample': 0.4187829352135062, 'colsample_bytree': 0.5525522630906509, 'min_child_weight': 2}. Best is trial 140 with value: 0.9545422235903054.
[I 2024-08-05 14:00:13,696] Trial 158 finished with value: 0.9526975701944551 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 917, 'learning_rate': 0.04482024274174625, 'max_depth': 3, 'subsample': 0.493

[I 2024-08-05 14:05:31,601] Trial 178 finished with value: 0.9520627742233634 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 921, 'learning_rate': 0.04659412870078104, 'max_depth': 3, 'subsample': 0.46581904996448187, 'colsample_bytree': 0.5686587936625962, 'min_child_weight': 2}. Best is trial 140 with value: 0.9545422235903054.
[I 2024-08-05 14:05:43,397] Trial 179 finished with value: 0.9521287294686499 and parameters: {'scaling_method': 'standard', 'encoding_method': 'ordinal', 'n_estimators': 893, 'learning_rate': 0.05134183906836873, 'max_depth': 3, 'subsample': 0.4251245355397435, 'colsample_bytree': 0.547097422191078, 'min_child_weight': 2}. Best is trial 140 with value: 0.9545422235903054.
[I 2024-08-05 14:06:00,423] Trial 180 finished with value: 0.9506661658006885 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'n_estimators': 938, 'learning_rate': 0.03631912680945246, 'max_depth': 3, 'subsample': 0.51823

In [17]:
print(study_xgb.best_value)
print(study_xgb.best_params)

0.9545422235903054
{'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 902, 'learning_rate': 0.04089478271640344, 'max_depth': 3, 'subsample': 0.4614417149387252, 'colsample_bytree': 0.6589253772701361, 'min_child_weight': 2}


In [18]:
trial_number = []
score = []
parameters = []
optuna_dict = {}
trials = range(200)

for trial in trials:
    trial_number.append(study_xgb.get_trials()[trial].number)
    score.append(study_xgb.get_trials()[trial].value)
    parameters.append(list(study_xgb.get_trials()[trial].params.items()))

optuna_dict['Trial'] = trial_number
optuna_dict['Score'] = score
optuna_dict['Parameters'] = parameters

optuna_xgb = pd.DataFrame.from_dict(optuna_dict)
optuna_xgb

,Trial,Score,Parameters
0,0,-1.380739,"[(scaling_method, standard), (encoding_method,..."
1,1,-6.566165,"[(scaling_method, maxabs), (encoding_method, o..."
2,2,0.919151,"[(scaling_method, maxabs), (encoding_method, o..."
3,3,-6.669372,"[(scaling_method, minmax), (encoding_method, o..."
4,4,0.948155,"[(scaling_method, maxabs), (encoding_method, o..."
...,...,...,...
195,195,0.951710,"[(scaling_method, standard), (encoding_method,..."
196,196,0.952328,"[(scaling_method, standard), (encoding_method,..."
197,197,0.951869,"[(scaling_method, standard), (encoding_method,..."
198,198,-4.566711,"[(scaling_method, standard), (encoding_method,..."


In [ ]:
pd.set_option('display.max_colwidth', None) 
pd.set_option('display.max_rows', None)
#optuna_xgb.sort_values(by='Score', ascending=False)

In [19]:
optuna_xgb['scaling'] =optuna_xgb.Parameters.apply(lambda x: x[0][1])
optuna_xgb['encoding'] = optuna_xgb.Parameters.apply(lambda x: x[1][1])
optuna_xgb['n_estimators'] = optuna_xgb.Parameters.apply(lambda x: x[-6][1])
optuna_xgb['learning_rate'] = optuna_xgb.Parameters.apply(lambda x: x[-5][1])
optuna_xgb['max_depth'] = optuna_xgb.Parameters.apply(lambda x: x[-4][1])
optuna_xgb['subsample'] = optuna_xgb.Parameters.apply(lambda x: x[-3][1])
optuna_xgb['colsample'] = optuna_xgb.Parameters.apply(lambda x: x[-2][1])
optuna_xgb['min_child'] = optuna_xgb.Parameters.apply(lambda x: x[-1][1])
optuna_xgb

,Trial,Score,Parameters,scaling,encoding,n_estimators,learning_rate,max_depth,subsample,colsample,min_child
0,0,-1.380739,"[(scaling_method, standard), (encoding_method,...",standard,onehot,342,0.008173,3,0.318556,0.554683,5
1,1,-6.566165,"[(scaling_method, maxabs), (encoding_method, o...",maxabs,onehot,629,0.001389,2,0.217026,0.692275,5
2,2,0.919151,"[(scaling_method, maxabs), (encoding_method, o...",maxabs,ordinal,119,0.071001,2,0.418680,0.813772,3
3,3,-6.669372,"[(scaling_method, minmax), (encoding_method, o...",minmax,ordinal,110,0.001042,2,0.129302,0.654093,5
4,4,0.948155,"[(scaling_method, maxabs), (encoding_method, o...",maxabs,onehot,765,0.040391,3,0.528579,0.981544,2
...,...,...,...,...,...,...,...,...,...,...,...
195,195,0.951710,"[(scaling_method, standard), (encoding_method,...",standard,onehot,905,0.036914,3,0.510848,0.535403,2
196,196,0.952328,"[(scaling_method, standard), (encoding_method,...",standard,onehot,839,0.046338,3,0.466496,0.529218,2
197,197,0.951869,"[(scaling_method, standard), (encoding_method,...",standard,onehot,865,0.043460,3,0.476313,0.573544,2
198,198,-4.566711,"[(scaling_method, standard), (encoding_method,...",standard,onehot,928,0.002009,3,0.562693,0.557418,2


In [46]:
optuna_xgb.max_depth = optuna_xgb.max_depth.apply(lambda x: str(x))

In [49]:
optuna_xgb.min_child = optuna_xgb.min_child.apply(lambda x: str(x))

In [48]:
optuna_xgb.to_csv('dataframes/optuna/optuna_xgb.csv')

In [41]:
fig = px.scatter(optuna_xgb.loc[optuna_xgb.Score > .93],
                 x="Trial", 
                 y="Score", 
                 color = 'learning_rate',
                 hover_data=['Parameters'],
                labels = {'Trial': 'Optuna Trial', 'Score': 'Score (R-Squared)'},
                title = 'XGB tuned with Optuna after 200 trials<br><sup> Best Score: 0.95454 </sup>')

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)
fig.write_image("images/optuna_xgb_learning_rate.png")
fig.show()

In [30]:
fig = px.scatter(optuna_xgb.loc[optuna_xgb.Score > .93],
                 x="Trial", 
                 y="Score", 
                 color = 'encoding',
                 hover_data=['Parameters'],
                labels = {'Trial': 'Optuna Trial', 'Score': 'Score (R-Squared)'},
                title = 'XGB tuned with Optuna after 200 trials<br><sup> Best Score: 0.95454 </sup>')

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)
fig.write_image("images/optuna_xgb_encoding.png")
fig.show()

In [43]:
fig = px.scatter(optuna_xgb.loc[optuna_xgb.Score > .93],
                 x="Trial", 
                 y="Score", 
                 color = 'n_estimators',
                 hover_data=['Parameters'],
                labels = {'Trial': 'Optuna Trial', 'Score': 'Score (R-Squared)'},
                title = 'XGB tuned with Optuna after 200 trials<br><sup> Best Score: 0.95454 </sup>')

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)
fig.write_image("images/optuna_xgb_nestimators.png")
fig.show()

In [52]:
fig = px.scatter(optuna_xgb.loc[optuna_xgb.Score > .93],
                 x="Trial", 
                 y="Score", 
                 color = 'min_child',
                 hover_data=['Parameters'],
                labels = {'Trial': 'Optuna Trial', 'Score': 'Score (R-Squared)'},
                title = 'XGB tuned with Optuna after 200 trials<br><sup> Best Score: 0.95454 </sup>')

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)
fig.write_image("images/optuna_xgb_minchild.png")
fig.show()

In [53]:
fig = px.scatter(optuna_xgb.loc[optuna_xgb.Score > .93],
                 x="Trial", 
                 y="Score", 
                 color = 'max_depth',
                 hover_data=['Parameters'],
                labels = {'Trial': 'Optuna Trial', 'Score': 'Score (R-Squared)'},
                title = 'XGB tuned with Optuna after 200 trials<br><sup> Best Score: 0.95454 </sup>')

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)
fig.write_image("images/optuna_xgb_maxdepth.png")
fig.show()

In [54]:
fig = px.scatter(optuna_xgb.loc[optuna_xgb.Score > .93],
                 x="Trial", 
                 y="Score", 
                 color = 'subsample',
                 hover_data=['Parameters'],
                labels = {'Trial': 'Optuna Trial', 'Score': 'Score (R-Squared)'},
                title = 'XGB tuned with Optuna after 200 trials<br><sup> Best Score: 0.95454 </sup>')

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)
fig.write_image("images/optuna_xgb_subsample.png")
fig.show()

In [55]:
fig = px.scatter(optuna_xgb.loc[optuna_xgb.Score > .93],
                 x="Trial", 
                 y="Score", 
                 color = 'colsample',
                 hover_data=['Parameters'],
                labels = {'Trial': 'Optuna Trial', 'Score': 'Score (R-Squared)'},
                title = 'XGB tuned with Optuna after 200 trials<br><sup> Best Score: 0.95454 </sup>')

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)
fig.write_image("images/optuna_xgb_colsample.png")
fig.show()

# RF

In [73]:
def instantiate_rf(trial : Trial) -> RandomForestRegressor:
    params = {
    'bootstrap':trial.suggest_categorical('bootstrap', [True]),
    'n_estimators': trial.suggest_int('n_estimators', 250, 500),
    'max_depth': trial.suggest_int('max_depth', 50, 75),
    'min_samples_split': trial.suggest_int('min_samples_split', 4, 6),
    'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5)
    }
    
    return RandomForestRegressor(**params)
  
def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor_ordinal(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_rf(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_rf(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [74]:
from optuna import create_study

study_rf = create_study(study_name='optimization', direction='maximize')

study_rf.optimize(lambda trial: objective_rf(trial, X, y), n_trials=200)

[I 2024-08-08 19:53:40,190] A new study created in memory with name: optimization
[I 2024-08-08 19:56:04,200] Trial 0 finished with value: 0.9084455125867825 and parameters: {'scaling_method': 'minmax', 'bootstrap': True, 'n_estimators': 304, 'max_depth': 70, 'min_samples_split': 5, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.9084455125867825.
[I 2024-08-08 19:58:14,972] Trial 1 finished with value: 0.9082270111188523 and parameters: {'scaling_method': 'robust', 'bootstrap': True, 'n_estimators': 254, 'max_depth': 50, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.9084455125867825.
[I 2024-08-08 19:59:44,927] Trial 2 finished with value: 0.9006897760382706 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 301, 'max_depth': 51, 'min_samples_split': 5, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.9084455125867825.
[I 2024-08-08 20:02:01,992] Trial 3 finished with value: 0.8998996415931397 and parameters: {'sc

[I 2024-08-08 21:13:27,628] Trial 30 finished with value: 0.9093670527110103 and parameters: {'scaling_method': 'robust', 'bootstrap': True, 'n_estimators': 294, 'max_depth': 53, 'min_samples_split': 4, 'min_samples_leaf': 2}. Best is trial 11 with value: 0.9098140035652329.
[I 2024-08-08 21:15:45,210] Trial 31 finished with value: 0.9081407157321943 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 318, 'max_depth': 66, 'min_samples_split': 4, 'min_samples_leaf': 3}. Best is trial 11 with value: 0.9098140035652329.
[I 2024-08-08 21:18:44,098] Trial 32 finished with value: 0.9068980764739286 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 353, 'max_depth': 66, 'min_samples_split': 4, 'min_samples_leaf': 2}. Best is trial 11 with value: 0.9098140035652329.
[I 2024-08-08 21:21:07,057] Trial 33 finished with value: 0.9084292129421894 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 252, 'max_depth

[I 2024-08-08 22:35:21,012] Trial 60 finished with value: 0.9072703895645693 and parameters: {'scaling_method': 'standard', 'bootstrap': True, 'n_estimators': 252, 'max_depth': 73, 'min_samples_split': 6, 'min_samples_leaf': 1}. Best is trial 39 with value: 0.9099364185081078.
[I 2024-08-08 22:37:58,214] Trial 61 finished with value: 0.9082667630838877 and parameters: {'scaling_method': 'standard', 'bootstrap': True, 'n_estimators': 274, 'max_depth': 71, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 39 with value: 0.9099364185081078.
[I 2024-08-08 22:40:25,317] Trial 62 finished with value: 0.9077690669972462 and parameters: {'scaling_method': 'standard', 'bootstrap': True, 'n_estimators': 259, 'max_depth': 67, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 39 with value: 0.9099364185081078.
[I 2024-08-08 22:43:27,395] Trial 63 finished with value: 0.9084370829842734 and parameters: {'scaling_method': 'standard', 'bootstrap': True, 'n_estimators': 314, 'm

[I 2024-08-09 00:02:55,333] Trial 90 finished with value: 0.9094966536496223 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 299, 'max_depth': 70, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 77 with value: 0.9119624967805391.
[I 2024-08-09 00:05:33,101] Trial 91 finished with value: 0.9080855336827329 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 268, 'max_depth': 66, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 77 with value: 0.9119624967805391.
[I 2024-08-09 00:08:13,304] Trial 92 finished with value: 0.9069188453037521 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 271, 'max_depth': 65, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 77 with value: 0.9119624967805391.
[I 2024-08-09 00:10:57,327] Trial 93 finished with value: 0.9098305207647642 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 279, 'max_depth

[I 2024-08-09 01:40:38,998] Trial 120 finished with value: 0.9099093882789652 and parameters: {'scaling_method': 'standard', 'bootstrap': True, 'n_estimators': 381, 'max_depth': 56, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 77 with value: 0.9119624967805391.
[I 2024-08-09 01:44:46,941] Trial 121 finished with value: 0.9070838087978907 and parameters: {'scaling_method': 'standard', 'bootstrap': True, 'n_estimators': 412, 'max_depth': 56, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 77 with value: 0.9119624967805391.
[I 2024-08-09 01:48:42,571] Trial 122 finished with value: 0.9080934999725387 and parameters: {'scaling_method': 'standard', 'bootstrap': True, 'n_estimators': 391, 'max_depth': 54, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 77 with value: 0.9119624967805391.
[I 2024-08-09 01:52:32,238] Trial 123 finished with value: 0.9085215517526632 and parameters: {'scaling_method': 'standard', 'bootstrap': True, 'n_estimators': 383

[I 2024-08-09 03:34:28,282] Trial 150 finished with value: 0.9080292681910649 and parameters: {'scaling_method': 'standard', 'bootstrap': True, 'n_estimators': 388, 'max_depth': 50, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 77 with value: 0.9119624967805391.
[I 2024-08-09 03:38:09,982] Trial 151 finished with value: 0.907483717038939 and parameters: {'scaling_method': 'standard', 'bootstrap': True, 'n_estimators': 373, 'max_depth': 51, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 77 with value: 0.9119624967805391.
[I 2024-08-09 03:42:10,329] Trial 152 finished with value: 0.9096394701444619 and parameters: {'scaling_method': 'standard', 'bootstrap': True, 'n_estimators': 399, 'max_depth': 55, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 77 with value: 0.9119624967805391.
[I 2024-08-09 03:46:03,006] Trial 153 finished with value: 0.909132745552292 and parameters: {'scaling_method': 'standard', 'bootstrap': True, 'n_estimators': 382, 

[I 2024-08-09 05:15:44,696] Trial 180 finished with value: 0.9090788454245444 and parameters: {'scaling_method': 'robust', 'bootstrap': True, 'n_estimators': 256, 'max_depth': 68, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 77 with value: 0.9119624967805391.
[I 2024-08-09 05:18:24,619] Trial 181 finished with value: 0.9085360866499472 and parameters: {'scaling_method': 'robust', 'bootstrap': True, 'n_estimators': 264, 'max_depth': 68, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 77 with value: 0.9119624967805391.
[I 2024-08-09 05:21:07,352] Trial 182 finished with value: 0.9117876770327767 and parameters: {'scaling_method': 'robust', 'bootstrap': True, 'n_estimators': 271, 'max_depth': 68, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 77 with value: 0.9119624967805391.
[I 2024-08-09 05:23:44,464] Trial 183 finished with value: 0.9100411342739213 and parameters: {'scaling_method': 'robust', 'bootstrap': True, 'n_estimators': 258, 'max_d

In [75]:
print(study_rf.best_value)
print(study_rf.best_params)

0.9119624967805391
{'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 320, 'max_depth': 69, 'min_samples_split': 4, 'min_samples_leaf': 1}


In [76]:
trial_number = []
score = []
parameters = []
optuna_dict = {}
trials = range(200)

for trial in trials:
    trial_number.append(study_rf.get_trials()[trial].number)
    score.append(study_rf.get_trials()[trial].value)
    parameters.append(list(study_rf.get_trials()[trial].params.items()))

optuna_dict['Trial'] = trial_number
optuna_dict['Score'] = score
optuna_dict['Parameters'] = parameters

optuna_rf = pd.DataFrame.from_dict(optuna_dict)
optuna_rf
optuna_rf.to_csv('final_project/dataframes/optuna/optuna_rf.csv')

# gbr

In [58]:
def instantiate_gbr(trial : Trial) -> GradientBoostingRegressor:
    params = {
    'n_estimators': trial.suggest_int('n_estimators', 500, 1000),
    'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1, log=True),
    'max_depth': trial.suggest_int ('max_depth', 2, 6),
    "subsample": trial.suggest_float("subsample", 0.5, 1.0),
    'min_samples_split': trial.suggest_int ('min_sample_split', 6, 10),
    'min_samples_leaf': trial.suggest_int ('min_sample_leaf', 1, 10)
    }
    
    return GradientBoostingRegressor(**params)

def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor_ordinal(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_gbr(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_gbr(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [59]:
from optuna import create_study

study_gbr = create_study(study_name='optimization', direction='maximize')

study_gbr.optimize(lambda trial: objective_gbr(trial, X, y), n_trials=200)

[I 2024-08-08 10:14:42,522] A new study created in memory with name: optimization
[I 2024-08-08 10:17:16,186] Trial 0 finished with value: 0.9357993555883761 and parameters: {'scaling_method': 'robust', 'n_estimators': 745, 'learning_rate': 0.21678069375759798, 'max_depth': 5, 'subsample': 0.8675314147132316, 'min_sample_split': 9, 'min_sample_leaf': 6}. Best is trial 0 with value: 0.9357993555883761.
[I 2024-08-08 10:19:25,495] Trial 1 finished with value: 0.9356945325081594 and parameters: {'scaling_method': 'standard', 'n_estimators': 927, 'learning_rate': 0.16676247629233104, 'max_depth': 5, 'subsample': 0.5346444711905138, 'min_sample_split': 10, 'min_sample_leaf': 5}. Best is trial 0 with value: 0.9357993555883761.
[I 2024-08-08 10:20:53,074] Trial 2 finished with value: 0.9404712778758498 and parameters: {'scaling_method': 'standard', 'n_estimators': 557, 'learning_rate': 0.01298555117298648, 'max_depth': 4, 'subsample': 0.8187148917834679, 'min_sample_split': 9, 'min_sample_lea

[I 2024-08-08 10:51:42,138] Trial 25 finished with value: 0.9321255339157309 and parameters: {'scaling_method': 'standard', 'n_estimators': 634, 'learning_rate': 0.005527375595333015, 'max_depth': 6, 'subsample': 0.5979671049367115, 'min_sample_split': 7, 'min_sample_leaf': 6}. Best is trial 20 with value: 0.9488677235690635.
[I 2024-08-08 10:52:54,581] Trial 26 finished with value: 0.9457853023708114 and parameters: {'scaling_method': 'robust', 'n_estimators': 779, 'learning_rate': 0.08508835771034046, 'max_depth': 3, 'subsample': 0.6926605798801646, 'min_sample_split': 7, 'min_sample_leaf': 9}. Best is trial 20 with value: 0.9488677235690635.
[I 2024-08-08 10:54:31,676] Trial 27 finished with value: 0.9439860119384299 and parameters: {'scaling_method': 'minmax', 'n_estimators': 671, 'learning_rate': 0.026960663311767864, 'max_depth': 4, 'subsample': 0.649923650841813, 'min_sample_split': 8, 'min_sample_leaf': 10}. Best is trial 20 with value: 0.9488677235690635.
[I 2024-08-08 10:55:3

[I 2024-08-08 11:27:30,522] Trial 51 finished with value: 0.9485288086331903 and parameters: {'scaling_method': 'standard', 'n_estimators': 681, 'learning_rate': 0.08643693942283374, 'max_depth': 3, 'subsample': 0.7757852337189054, 'min_sample_split': 7, 'min_sample_leaf': 8}. Best is trial 20 with value: 0.9488677235690635.
[I 2024-08-08 11:28:28,581] Trial 52 finished with value: 0.9476362634154034 and parameters: {'scaling_method': 'standard', 'n_estimators': 650, 'learning_rate': 0.03833007516940945, 'max_depth': 3, 'subsample': 0.6940837531524914, 'min_sample_split': 7, 'min_sample_leaf': 7}. Best is trial 20 with value: 0.9488677235690635.
[I 2024-08-08 11:29:45,324] Trial 53 finished with value: 0.9451518125715406 and parameters: {'scaling_method': 'standard', 'n_estimators': 684, 'learning_rate': 0.023922287171592217, 'max_depth': 3, 'subsample': 0.9361692598866286, 'min_sample_split': 7, 'min_sample_leaf': 8}. Best is trial 20 with value: 0.9488677235690635.
[I 2024-08-08 11:3

[I 2024-08-08 12:07:26,725] Trial 77 finished with value: 0.9482938507896718 and parameters: {'scaling_method': 'minmax', 'n_estimators': 974, 'learning_rate': 0.0301527170697918, 'max_depth': 3, 'subsample': 0.7225652521396553, 'min_sample_split': 7, 'min_sample_leaf': 2}. Best is trial 61 with value: 0.9519824147293688.
[I 2024-08-08 12:08:37,428] Trial 78 finished with value: 0.9497105734369352 and parameters: {'scaling_method': 'minmax', 'n_estimators': 929, 'learning_rate': 0.07022076278354103, 'max_depth': 2, 'subsample': 0.7475225262005804, 'min_sample_split': 7, 'min_sample_leaf': 2}. Best is trial 61 with value: 0.9519824147293688.
[I 2024-08-08 12:10:18,346] Trial 79 finished with value: 0.9466249587091303 and parameters: {'scaling_method': 'minmax', 'n_estimators': 985, 'learning_rate': 0.11417193772893686, 'max_depth': 3, 'subsample': 0.6591564374194722, 'min_sample_split': 7, 'min_sample_leaf': 1}. Best is trial 61 with value: 0.9519824147293688.
[I 2024-08-08 12:11:58,787

[I 2024-08-08 12:50:39,042] Trial 103 finished with value: 0.9493064590934024 and parameters: {'scaling_method': 'minmax', 'n_estimators': 894, 'learning_rate': 0.033963737383036895, 'max_depth': 3, 'subsample': 0.7171837268042968, 'min_sample_split': 8, 'min_sample_leaf': 4}. Best is trial 61 with value: 0.9519824147293688.
[I 2024-08-08 12:52:19,913] Trial 104 finished with value: 0.43234261529324103 and parameters: {'scaling_method': 'minmax', 'n_estimators': 866, 'learning_rate': 0.0006171518718191068, 'max_depth': 3, 'subsample': 0.76121489664128, 'min_sample_split': 8, 'min_sample_leaf': 2}. Best is trial 61 with value: 0.9519824147293688.
[I 2024-08-08 12:54:01,765] Trial 105 finished with value: 0.9481219811152698 and parameters: {'scaling_method': 'minmax', 'n_estimators': 922, 'learning_rate': 0.019737534067039018, 'max_depth': 3, 'subsample': 0.7286939095203531, 'min_sample_split': 8, 'min_sample_leaf': 2}. Best is trial 61 with value: 0.9519824147293688.
[I 2024-08-08 12:55

[I 2024-08-08 13:33:51,771] Trial 129 finished with value: 0.9513593245014202 and parameters: {'scaling_method': 'standard', 'n_estimators': 906, 'learning_rate': 0.047307177345022204, 'max_depth': 3, 'subsample': 0.6853170034906759, 'min_sample_split': 8, 'min_sample_leaf': 4}. Best is trial 125 with value: 0.9521730810246176.
[I 2024-08-08 13:35:21,423] Trial 130 finished with value: 0.9504882291608991 and parameters: {'scaling_method': 'standard', 'n_estimators': 894, 'learning_rate': 0.04516463650362059, 'max_depth': 3, 'subsample': 0.6488020950557758, 'min_sample_split': 8, 'min_sample_leaf': 4}. Best is trial 125 with value: 0.9521730810246176.
[I 2024-08-08 13:36:52,379] Trial 131 finished with value: 0.9512642057037475 and parameters: {'scaling_method': 'standard', 'n_estimators': 917, 'learning_rate': 0.045293333563038926, 'max_depth': 3, 'subsample': 0.6497176811488974, 'min_sample_split': 8, 'min_sample_leaf': 4}. Best is trial 125 with value: 0.9521730810246176.
[I 2024-08-

[I 2024-08-08 14:13:52,136] Trial 154 finished with value: 0.9492743079836483 and parameters: {'scaling_method': 'standard', 'n_estimators': 926, 'learning_rate': 0.024895039022961477, 'max_depth': 3, 'subsample': 0.6747628503227319, 'min_sample_split': 8, 'min_sample_leaf': 6}. Best is trial 125 with value: 0.9521730810246176.
[I 2024-08-08 14:15:28,594] Trial 155 finished with value: 0.9507268705601373 and parameters: {'scaling_method': 'standard', 'n_estimators': 936, 'learning_rate': 0.033764078261197955, 'max_depth': 3, 'subsample': 0.6805224565748115, 'min_sample_split': 8, 'min_sample_leaf': 6}. Best is trial 125 with value: 0.9521730810246176.
[I 2024-08-08 14:17:04,465] Trial 156 finished with value: 0.9521478279916111 and parameters: {'scaling_method': 'standard', 'n_estimators': 952, 'learning_rate': 0.03880500805924664, 'max_depth': 3, 'subsample': 0.6554918913604734, 'min_sample_split': 8, 'min_sample_leaf': 5}. Best is trial 125 with value: 0.9521730810246176.
[I 2024-08-

[I 2024-08-08 14:54:26,304] Trial 179 finished with value: 0.9486272022059673 and parameters: {'scaling_method': 'standard', 'n_estimators': 936, 'learning_rate': 0.018479125516759815, 'max_depth': 3, 'subsample': 0.6955809970209594, 'min_sample_split': 8, 'min_sample_leaf': 6}. Best is trial 125 with value: 0.9521730810246176.
[I 2024-08-08 14:56:00,385] Trial 180 finished with value: 0.9505629201281021 and parameters: {'scaling_method': 'standard', 'n_estimators': 922, 'learning_rate': 0.03716932227269911, 'max_depth': 3, 'subsample': 0.6607655823251474, 'min_sample_split': 8, 'min_sample_leaf': 5}. Best is trial 125 with value: 0.9521730810246176.
[I 2024-08-08 14:57:37,041] Trial 181 finished with value: 0.9515148528899207 and parameters: {'scaling_method': 'standard', 'n_estimators': 926, 'learning_rate': 0.03980953608095004, 'max_depth': 3, 'subsample': 0.6804865815697551, 'min_sample_split': 8, 'min_sample_leaf': 6}. Best is trial 125 with value: 0.9521730810246176.
[I 2024-08-0

In [60]:
print(study_gbr.best_value)
print(study_gbr.best_params)

0.9521730810246176
{'scaling_method': 'minmax', 'n_estimators': 856, 'learning_rate': 0.04767898601143354, 'max_depth': 3, 'subsample': 0.68704242814767, 'min_sample_split': 8, 'min_sample_leaf': 3}


In [61]:
trial_number = []
score = []
parameters = []
optuna_dict = {}
trials = range(200)

for trial in trials:
    trial_number.append(study_gbr.get_trials()[trial].number)
    score.append(study_gbr.get_trials()[trial].value)
    parameters.append(list(study_gbr.get_trials()[trial].params.items()))

optuna_dict['Trial'] = trial_number
optuna_dict['Score'] = score
optuna_dict['Parameters'] = parameters

optuna_gbr = pd.DataFrame.from_dict(optuna_dict)
optuna_gbr


,Trial,Score,Parameters
0,0,0.935799,"[(scaling_method, robust), (n_estimators, 745)..."
1,1,0.935695,"[(scaling_method, standard), (n_estimators, 92..."
2,2,0.940471,"[(scaling_method, standard), (n_estimators, 55..."
3,3,0.937434,"[(scaling_method, robust), (n_estimators, 739)..."
4,4,0.933467,"[(scaling_method, maxabs), (n_estimators, 839)..."
...,...,...,...
195,195,0.948768,"[(scaling_method, standard), (n_estimators, 91..."
196,196,0.950713,"[(scaling_method, standard), (n_estimators, 96..."
197,197,0.949612,"[(scaling_method, standard), (n_estimators, 88..."
198,198,0.951345,"[(scaling_method, standard), (n_estimators, 93..."


In [63]:
optuna_gbr.to_csv('final_project/dataframes/optuna/optuna_gbr.csv')

# Adaboost

In [64]:
def instantiate_ada(trial : Trial) -> AdaBoostRegressor:
    params = {
    'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1, log=True),
    'loss': trial.suggest_categorical('loss',['linear', 'square', 'exponential']),
    'n_estimators': trial.suggest_int('n_estimators', 1, 1000)
    }
    
    return AdaBoostRegressor(**params)

def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor_ordinal(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_ada(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_ada(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [65]:
from optuna import create_study

study_ada = create_study(study_name='optimization', direction='maximize')

study_ada.optimize(lambda trial: objective_ada(trial, X, y), n_trials=200)

[I 2024-08-08 15:51:37,983] A new study created in memory with name: optimization
[I 2024-08-08 15:52:44,990] Trial 0 finished with value: 0.7576802333662138 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.0012834671890213096, 'loss': 'exponential', 'n_estimators': 574}. Best is trial 0 with value: 0.7576802333662138.
[I 2024-08-08 15:53:46,952] Trial 1 finished with value: 0.877585271183217 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.9023810892585588, 'loss': 'linear', 'n_estimators': 846}. Best is trial 1 with value: 0.877585271183217.
[I 2024-08-08 15:53:57,145] Trial 2 finished with value: 0.8571958861413098 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.3301939662166686, 'loss': 'square', 'n_estimators': 103}. Best is trial 1 with value: 0.877585271183217.
[I 2024-08-08 15:55:49,103] Trial 3 finished with value: 0.8005036506616587 and parameters: {'scaling_method': 'minmax', 'learning_rate': 0.004022998111875487, 'loss': 'line

[I 2024-08-08 16:24:25,396] Trial 33 finished with value: 0.8778980298279404 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.9803591244167049, 'loss': 'linear', 'n_estimators': 791}. Best is trial 17 with value: 0.8781365344498185.
[I 2024-08-08 16:25:31,282] Trial 34 finished with value: 0.8756807721646516 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.4822146896731688, 'loss': 'linear', 'n_estimators': 781}. Best is trial 17 with value: 0.8781365344498185.
[I 2024-08-08 16:26:17,687] Trial 35 finished with value: 0.8770311964266357 and parameters: {'scaling_method': 'minmax', 'learning_rate': 0.9819953349158357, 'loss': 'linear', 'n_estimators': 558}. Best is trial 17 with value: 0.8781365344498185.
[I 2024-08-08 16:27:12,828] Trial 36 finished with value: 0.8730369858134848 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.545186468501077, 'loss': 'square', 'n_estimators': 660}. Best is trial 17 with value: 0.8781365344498185.
[I 2024-0

[I 2024-08-08 17:00:53,053] Trial 66 finished with value: 0.8775128966795599 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.9745020058128762, 'loss': 'linear', 'n_estimators': 878}. Best is trial 62 with value: 0.8793962498473966.
[I 2024-08-08 17:02:16,039] Trial 67 finished with value: 0.876278436529363 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.6437002291635945, 'loss': 'linear', 'n_estimators': 957}. Best is trial 62 with value: 0.8793962498473966.
[I 2024-08-08 17:03:30,708] Trial 68 finished with value: 0.8765647016754828 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.4460786155397767, 'loss': 'exponential', 'n_estimators': 826}. Best is trial 62 with value: 0.8793962498473966.
[I 2024-08-08 17:04:54,832] Trial 69 finished with value: 0.869313164743394 and parameters: {'scaling_method': 'minmax', 'learning_rate': 0.10196283515738638, 'loss': 'linear', 'n_estimators': 772}. Best is trial 62 with value: 0.8793962498473966.
[I 2

[I 2024-08-08 17:36:52,293] Trial 99 finished with value: 0.8771267003642997 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.793802766264144, 'loss': 'linear', 'n_estimators': 849}. Best is trial 62 with value: 0.8793962498473966.
[I 2024-08-08 17:37:56,526] Trial 100 finished with value: 0.8762919313272896 and parameters: {'scaling_method': 'robust', 'learning_rate': 0.5218126516357873, 'loss': 'linear', 'n_estimators': 969}. Best is trial 62 with value: 0.8793962498473966.
[I 2024-08-08 17:38:50,119] Trial 101 finished with value: 0.8775513902609691 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.9793833990285896, 'loss': 'linear', 'n_estimators': 861}. Best is trial 62 with value: 0.8793962498473966.
[I 2024-08-08 17:39:49,583] Trial 102 finished with value: 0.8762327048017976 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.6866025944494901, 'loss': 'linear', 'n_estimators': 934}. Best is trial 62 with value: 0.8793962498473966.
[I 202

[I 2024-08-08 18:12:45,184] Trial 132 finished with value: 0.8762874792212815 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.7793228900738646, 'loss': 'exponential', 'n_estimators': 889}. Best is trial 123 with value: 0.879528195262105.
[I 2024-08-08 18:13:57,462] Trial 133 finished with value: 0.876745820964311 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.6012232807897382, 'loss': 'exponential', 'n_estimators': 922}. Best is trial 123 with value: 0.879528195262105.
[I 2024-08-08 18:15:03,998] Trial 134 finished with value: 0.8779063231516251 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.8489337943175218, 'loss': 'exponential', 'n_estimators': 875}. Best is trial 123 with value: 0.879528195262105.
[I 2024-08-08 18:16:13,874] Trial 135 finished with value: 0.8775955016587297 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.9885299401124495, 'loss': 'square', 'n_estimators': 949}. Best is trial 123 with value: 0.8795281

[I 2024-08-08 18:44:16,196] Trial 165 finished with value: 0.8758497787779665 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.5923532864282013, 'loss': 'linear', 'n_estimators': 571}. Best is trial 123 with value: 0.879528195262105.
[I 2024-08-08 18:44:53,656] Trial 166 finished with value: 0.876994974318053 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.7080395471095342, 'loss': 'linear', 'n_estimators': 486}. Best is trial 123 with value: 0.879528195262105.
[I 2024-08-08 18:45:34,750] Trial 167 finished with value: 0.8767426320090643 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.9990570517266106, 'loss': 'exponential', 'n_estimators': 517}. Best is trial 123 with value: 0.879528195262105.
[I 2024-08-08 18:46:11,668] Trial 168 finished with value: 0.8749068082666209 and parameters: {'scaling_method': 'standard', 'learning_rate': 0.8499933485735998, 'loss': 'square', 'n_estimators': 491}. Best is trial 123 with value: 0.879528195

[I 2024-08-08 19:20:35,130] Trial 198 finished with value: 0.8776750346311152 and parameters: {'scaling_method': 'minmax', 'learning_rate': 0.9891430565894365, 'loss': 'exponential', 'n_estimators': 979}. Best is trial 123 with value: 0.879528195262105.
[I 2024-08-08 19:21:47,153] Trial 199 finished with value: 0.8778727155426795 and parameters: {'scaling_method': 'maxabs', 'learning_rate': 0.8244454580825864, 'loss': 'linear', 'n_estimators': 948}. Best is trial 123 with value: 0.879528195262105.


In [66]:
print(study_ada.best_value)
print(study_ada.best_params)

0.879528195262105
{'scaling_method': 'maxabs', 'learning_rate': 0.9958958810824097, 'loss': 'linear', 'n_estimators': 988}


In [67]:
trial_number = []
score = []
parameters = []
optuna_dict = {}
trials = range(200)

for trial in trials:
    trial_number.append(study_ada.get_trials()[trial].number)
    score.append(study_ada.get_trials()[trial].value)
    parameters.append(list(study_ada.get_trials()[trial].params.items()))

optuna_dict['Trial'] = trial_number
optuna_dict['Score'] = score
optuna_dict['Parameters'] = parameters

optuna_ada = pd.DataFrame.from_dict(optuna_dict)

optuna_ada

,Trial,Score,Parameters
0,0,0.757680,"[(scaling_method, maxabs), (learning_rate, 0.0..."
1,1,0.877585,"[(scaling_method, robust), (learning_rate, 0.9..."
2,2,0.857196,"[(scaling_method, standard), (learning_rate, 0..."
3,3,0.800504,"[(scaling_method, minmax), (learning_rate, 0.0..."
4,4,0.842198,"[(scaling_method, minmax), (learning_rate, 0.0..."
...,...,...,...
195,195,0.878719,"[(scaling_method, robust), (learning_rate, 0.9..."
196,196,0.877953,"[(scaling_method, maxabs), (learning_rate, 0.8..."
197,197,0.877462,"[(scaling_method, robust), (learning_rate, 0.7..."
198,198,0.877675,"[(scaling_method, minmax), (learning_rate, 0.9..."


In [69]:
optuna_ada.to_csv('final_project/dataframes/optuna/optuna_ada.csv')